## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-12-16-55-11 +0000,startribune,The latest: Immigration officers out in force ...,https://www.startribune.com/ice-raids-minnesot...
1,2026-01-12-16-52-07 +0000,cbc,Wiretaps caught Colombian cocaine group discus...,https://www.cbc.ca/news/world/cartel-suns-vene...
2,2026-01-12-16-51-12 +0000,nypost,Major NYC hospital accuses nurses’ union of pr...,https://nypost.com/2026/01/12/us-news/major-ny...
3,2026-01-12-16-49-21 +0000,nyt,Paramount Threatens Board Fight Over Warner Br...,https://www.nytimes.com/2026/01/12/business/me...
4,2026-01-12-16-45-00 +0000,startribune,MPR’s Angela Davis announces she is cancer-free,https://www.startribune.com/mprs-angela-davis-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2348/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
51,trump,32
95,iran,23
75,powell,19
47,fed,16
126,chair,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
135,2026-01-12-05-56-00 +0000,wsj,The criminal investigation into Federal Reserv...,https://www.wsj.com/economy/central-banking/tr...,111
109,2026-01-12-10-30-00 +0000,wsj,"For years, Fed Chair Jerome Powell responded t...",https://www.wsj.com/economy/central-banking/fo...,110
82,2026-01-12-12-03-00 +0000,wsj,How the U.S. and Iran went back to the brink o...,https://www.wsj.com/politics/policy/how-the-u-...,109
7,2026-01-12-16-36-01 +0000,nypost,GOP Sen. Thom Tillis says he will block any of...,https://nypost.com/2026/01/12/business/gop-sen...,106
90,2026-01-12-11-32-54 +0000,nypost,Trump addresses bombshell Jerome Powell invest...,https://nypost.com/2026/01/12/us-news/trump-sa...,102


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
135,111,2026-01-12-05-56-00 +0000,wsj,The criminal investigation into Federal Reserv...,https://www.wsj.com/economy/central-banking/tr...
138,85,2026-01-12-05-36-07 +0000,bbc,Trump mulls 'very strong' military options as ...,https://www.bbc.com/news/articles/czdqy3rmp78o...
52,55,2026-01-12-14-10-00 +0000,wsj,Mary Peltola will challenge GOP incumbent Sen....,https://www.wsj.com/politics/elections/mary-pe...
41,40,2026-01-12-14-33-37 +0000,cbc,"Malaysia, Indonesia become 1st countries to bl...",https://www.cbc.ca/news/business/malaysia-indo...
129,35,2026-01-12-08-13-29 +0000,wsj,"Stock Market Today: Nasdaq Edges Higher, Gold ...",https://www.wsj.com/livecoverage/stock-market-...
62,34,2026-01-12-13-24-31 +0000,cbc,Swiss court keeps bar owner in custody after d...,https://www.cbc.ca/news/world/swiss-court-bar-...
104,34,2026-01-12-10-43-00 +0000,bbc,He once criticised African leaders who cling t...,https://www.bbc.com/news/articles/cr7jep39eg4o...
2,32,2026-01-12-16-51-12 +0000,nypost,Major NYC hospital accuses nurses’ union of pr...,https://nypost.com/2026/01/12/us-news/major-ny...
149,31,2026-01-12-03-16-59 +0000,startribune,Minnesota’s Sun Country Airlines to be acquire...,https://www.startribune.com/sun-country-allegi...
89,30,2026-01-12-11-50-09 +0000,nypost,Gut-wrenching footage shows grieving Iranian f...,https://nypost.com/2026/01/12/world-news/foota...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
